# Use pretrained ResNet cnn to predict concentration plot over 6-minute sampling
### Firstly, import dependencies

In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import os
from PIL import Image
cudnn.benchmark = True
plt.ion()   # interactive mode

### Then, we import the pre-fine-tuned model (a .pth file), and pass it to gpu memory.

In [2]:
model = torchvision.models.alexnet(pretrained=False)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # 假设有2个分类

# 加载模型权重
model.load_state_dict(torch.load('best_alexnet.pth'))
model.eval()
model.cuda()                                                    # pass the model to cuda                                                 # test the model-loading condition

/home/cybercricetus/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cybercricetus/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### We define some pre-process transformation for the test images.

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224), transforms.InterpolationMode.BILINEAR),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224), transforms.InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}           # ResNet only support 224*224 images, resize to avoid losing information

### Define device variable (optional)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Define directory and classes

In [5]:
_dir = "tests"
_classes = ['near', 'far']
pic_name_list = _1st_dir = os.listdir(_dir)

### Traverse through the directory, read the images to CPU buffer and then GPU buffer, and finally do prediction with model loaded in GPU memory

In [6]:
count = 0                                               # correct counts
result_cmp_set = {}                                     # actual:pred
for name in pic_name_list:                              # traverse the directory
    img = Image.open(_dir+'/'+name).convert('RGB')      # open the image
    img = data_transforms['val'](img)                   # do the transformation
    img = img.unsqueeze(0)                      
    img = img.to(device)                                # pass to GPU memory

    with torch.no_grad():           
        outputs = model(img)                            # predict
        _, preds = torch.max(outputs, 1)
    result_cmp_set[name] = _classes[preds]              # add prediction to the collection
    if _classes[preds][0] == name[1]:                   # if correctly predicted, count+1
        count += 1

### Check the result 

In [7]:
print(result_cmp_set)                                 
print('accuracy is: ', count / len(pic_name_list))   # accuracy
print("Correct count = ", count)
print("Incorrect count = ", len(pic_name_list)-count)

{'_far6.png': 'far', '_near3.png': 'near', '_far5.png': 'far', '_far2.png': 'far', '_far8.png': 'far', '_far3.png': 'far', '_near6.png': 'near', '_far9.png': 'far', '_near9.png': 'near', '_near7.png': 'near', '_far10.png': 'far', '_far7.png': 'far', '_near2.png': 'near', '_far4.png': 'far', '_near4.png': 'far', '_near5.png': 'near', '_near8.png': 'near', '_far1.png': 'far', '_near1.png': 'near', '_near10.png': 'far'}
accuracy is:  0.9
Correct count =  18
Incorrect count =  2
